## Metadata VDB Search
We will use Milvus and Towhee to help searches. Towhee is used to extract the semantics of the text and return the text embedding. The Milvus vector database can store and search vectors, and return related dataset's metadata. So we first need to install [Milvus](https://github.com/milvus-io/milvus) and [Towhee](https://github.com/towhee-io/towhee).

Before getting started, please make sure that you have started a [Milvus service](https://milvus.io/docs/install_standalone-docker.md). This notebook uses [milvus 2.2.10](https://milvus.io/docs/v2.2.x/install_standalone-docker.md) and [pymilvus 2.2.11](https://milvus.io/docs/release_notes.md#2210).

<br>
<br>

`Things to look into:`
1. Are there better encoding models for Norwegian text than 'facebook/dpr-ctx_encoder-single-nq-base'?
2. What alternatives are there to RAG for search?
3. What working alternatives have already been made by others (don't reinvent the wheel)?

### Package installations

In [ ]:
#! pip install --upgrade pip
#! pip3 install -q towhee pymilvus==2.2.11
#! pip3 uninstall pymilvus -y

! pip3 install -q towhee pymilvus==2.1.1
! pip install transformers -q
! pip install pandas -q
! pip install tqdm -q
! pip3 show pymilvus ', ' grep -Ei 'Name:', Version:'
! pip3 show towhee | grep -Ei 'Name:|Version:'
! pip3 show transformers | grep -Ei 'Name:|Version:'

## 1.1 Adding embeddings for columns

The dataset is from the [Kartverket dataset metadata](https://cdn.discordapp.com/attachments/1204433663035449384/1206537816654356480/metadata_no_format.csv?ex=65dc5ee7&is=65c9e9e7&hm=3b9a88db41103ef5393294c5eaeebb60ee2229f43724cc014d4cffc92de1f384&), which contains metadata about each dataset.

The strings in the columns need to be converted to vector representations (embedding) using Towhee [text_embedding.dpr operator](https://towhee.io/text-embedding/dpr). Columns containing these new embedings should contain the original column name with `_vector` at the end.

## 1.2 Clean dataset for special characters, and output csv

In [45]:
import pandas as pd

# Clean the 'abstract' column for weird substrings
def clean_abstract(text):
    if pd.isnull(text):
        return text
    text = text.replace('"', '').replace('\\', '')

    return text

# Replaces special characters on text columns
def clean_special_substrings(text):
    # Removes special characters and weird substrings
    text = text.replace('  v/ ', ' ').replace('\\n', ' ').replace('▬', '').replace('\u00A0', ' ')

    return text

def clean_hash_delimiter(text):
    text = text.replace('###', ',')

    return text

def clean_constraints_column(text):
    if text == '###':
        text = text.replace('###', '')
    else:
        text = text.replace('###', ',')
    
    return text

def clean_security_constraints(text):
    text = text.replace('#########', '')

    return text

def clean_legal_constraints(text):
    text = text.replace('######', ',').replace('###', ',')

    return text

def clean_contact(text):
    text = text.replace('###', ',')

    return text

def clean_links(text):
    if text[:3] == '###':
        text = text[3:]
    
    return text

def clean_product_info(text):
    text = text.replace('###', ',')

    return text


dataset_file = 'Metadata_excel.xlsx'
df = pd.read_excel(dataset_file)

# Fill NaN values with an empty string
df.fillna('', inplace=True)

columns_clean_special_chars = [
    'schema', 'uuid', 'hierarchyLevel', 'title', 'abstract', 'keyword', 'geoBox', 'Constraints', 
    'SecurityConstraints', 'LegalConstraints', 'temporalExtent', 'responsibleParty',  
    'productInformation', 'parentId'
]
# Apply cleaning functions columns
for col in df.columns:
    # Apply cleaning to all string columns
    if df[col].dtype == 'object':
        if col in columns_clean_special_chars:
            df[col] = df[col].apply(clean_special_substrings)
            #df[col] = df[col].apply(replace_norwegian_characters)

        # Include cleaning of weird substrings
        if col == 'abstract':
            df[col] = df[col].apply(clean_abstract)
        
        # Clean keyword column
        if col == 'keyword':
            df[col] = df[col].apply(clean_hash_delimiter)

        # Clean geoBox column
        if col == 'geoBox':
            df[col] = df[col].apply(clean_hash_delimiter)

         # Clean Constraints column
        if col == 'Constraints':
            df[col] = df[col].apply(clean_constraints_column)
       
        # Clean SecurityConstraints column
        if col == 'SecurityConstraints':
            df[col] = df[col].apply(clean_security_constraints)
 
        # Clean LegalConstraints column
        if col == 'LegalConstraints':
            df[col] = df[col].apply(clean_legal_constraints)
        
        # Clean LegalConstraints column
        if col == 'responsibleParty':
            df[col] = df[col].apply(clean_contact)

        # Clean productInformation column
        if col == 'productInformation':
            df[col] = df[col].apply(clean_product_info)

        # Clean links columns
        if col in ['image', 'link']:
            df[col] = df[col].apply(clean_links)

 
cleaned_csv_file = 'output_metadata.csv'
df.to_csv(cleaned_csv_file, sep='|', index=False)
df.head()

## 1.3 Split a column into sentences using NLTK for better RAG performance

In [ ]:
! pip3 install nltk -q
! pip3 show nltk | grep -Ei 'Name:|Version:'

In [ ]:
import nltk
nltk.download('punkt')

In [47]:
import pandas as pd
from nltk.tokenize import sent_tokenize


# Function to split text into sentences and replicate row data
def split_into_sentences(row, text_column):
    sentences = sent_tokenize(row[text_column], language='norwegian')
    return [row.to_dict() for _ in sentences], sentences


# Loads dataset, and expands row into copies for each sentence in column
df = pd.read_csv(cleaned_csv_file, sep='|')
column_to_expand = 'abstract'
expanded_rows = df.apply(lambda row: split_into_sentences(row, column_to_expand), axis=1)

# Separate metadata (the row copy) and sentences, then flatten the lists
metadata, sentences = zip(*expanded_rows)
metadata = [item for sublist in metadata for item in sublist]
sentences = [sentence for sublist in sentences for sentence in sublist]

# Create a new DataFrame with expanded rows
sentence_column = 'sentence'
new_df = pd.DataFrame(metadata)
new_df[sentence_column] = sentences

expanded_dataset_file = 'cleaned_sentence_segmented_metadata.csv'
new_df.to_csv(expanded_dataset_file, sep='|', index=False)
new_df.head()

,schema,uuid,id,hierarchyLevel,title,datasetcreationdate,abstract,keyword,geoBox,Constraints,SecurityConstraints,LegalConstraints,temporalExtent,image,responsibleParty,link,metadatacreationdate,productInformation,parentId,sentence
0,iso19139,7a62f16f-9aeb-4c39-bf5f-e710232fa366,37228,software,Artsfunn,NaN,Datasettet inneholder stedfestet informasjon a...,"Natur,Norge,Svalbard,lav,karplanter,botanikkda...","2,33,57,81",NaN,NaN,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://editor.geonorge.no/thumbnails/7a62f16f...,Frank HansenNorsk institutt for naturforskning...,https://ipt.nina.no/,2021-03-24,"Produktspesifikasjon,Produktark,Produktside,Te...",NaN,Datasettet inneholder stedfestet informasjon a...
1,iso19139,7a62f16f-9aeb-4c39-bf5f-e710232fa366,37228,software,Artsfunn,NaN,Datasettet inneholder stedfestet informasjon a...,"Natur,Norge,Svalbard,lav,karplanter,botanikkda...","2,33,57,81",NaN,NaN,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://editor.geonorge.no/thumbnails/7a62f16f...,Frank HansenNorsk institutt for naturforskning...,https://ipt.nina.no/,2021-03-24,"Produktspesifikasjon,Produktark,Produktside,Te...",NaN,Funndataene følger datastandarden Darwin Core ...
2,iso19139,79013154-92ee-4647-b160-925cbc148601,21400,dataset,Hav og is - Iskart (shapefil),NaN,Istjenesten ved Meteorologisk institutt utarbe...,"Oceanographic geographical features,Inspire,No...","2.00,33.00,57.00,72.00",Bruksbegrensninger Ingen begrensninger på bruk...,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,"Meteorologisk instituttistjenesten@met.no,Mete...",http://polarview.met.no/,2023-11-15,"Produktspesifikasjon,Produktark,Produktside,Te...",NaN,Istjenesten ved Meteorologisk institutt utarbe...
3,iso19139,79013154-92ee-4647-b160-925cbc148601,21400,dataset,Hav og is - Iskart (shapefil),NaN,Istjenesten ved Meteorologisk institutt utarbe...,"Oceanographic geographical features,Inspire,No...","2.00,33.00,57.00,72.00",Bruksbegrensninger Ingen begrensninger på bruk...,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,"Meteorologisk instituttistjenesten@met.no,Mete...",http://polarview.met.no/,2023-11-15,"Produktspesifikasjon,Produktark,Produktside,Te...",NaN,Kartene viser iskonsentrasjon og er basert på ...
4,iso19139,79013154-92ee-4647-b160-925cbc148601,21400,dataset,Hav og is - Iskart (shapefil),NaN,Istjenesten ved Meteorologisk institutt utarbe...,"Oceanographic geographical features,Inspire,No...","2.00,33.00,57.00,72.00",Bruksbegrensninger Ingen begrensninger på bruk...,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,"Meteorologisk instituttistjenesten@met.no,Mete...",http://polarview.met.no/,2023-11-15,"Produktspesifikasjon,Produktark,Produktside,Te...",NaN,I tillegg til disse dataene brukes visuelle og...


## 1.4 Load dataset and vectorise chosen column

In [86]:
import pandas as pd
from towhee import pipe, ops, DataCollection
from transformers import AutoTokenizer
from tqdm import tqdm

# Function to compute embeddings for a single text
def compute_embeddings(text):
    MAX_TOKENS = 512 
    inputs = tokenizer(text, return_tensors="pt", max_length=MAX_TOKENS, truncation=True)
    truncated_text = tokenizer.decode(inputs["input_ids"][0])
    return DataCollection(embeddings_pipe(truncated_text)).to_list()[0]['vec']


# Loads dataset into dataframe and recasts columns into correct datatypes
# Either use cleaned_dataset, or cleaned_sentence_expanded datasets
df_kartverket = pd.read_excel(expanded_dataset_file)
recast_to_string = ['datasetcreationdate', 'metadatacreationdate']
df_kartverket[recast_to_string] = df_kartverket[recast_to_string].astype('object')

# Fill NaN values with an empty string
df_kartverket.fillna('', inplace=True)

# Pipe converting text to embeddings (vectors) using a model
facebook_context_model_name = 'facebook/dpr-ctx_encoder-single-nq-base'
facebook_question_model_name = 'facebook/dpr-question_encoder-single-nq-base'
mbert_model_name = 'bert-base-multilingual-uncased'
gpt_2_model_name = 'gpt2-xl'
openai_davinci_model_name = 'text-similarity-davinci-001' # requires password/username
bert_t5_model_name = 'sentence-t5-xxl'
norT5_model_name = 'ltg/nort5-large'
norBert_model_name = 'ltg/norbert3-base'

chosen_model = facebook_context_model_name
tokenizer = AutoTokenizer.from_pretrained(chosen_model)
embeddings_pipe = (
    pipe.input('text')
        .map('text', 'vec', ops.text_embedding.dpr(model_name=chosen_model))
        .output('vec')
)

# Process each column and create new columns for embeddings
column_to_vectorise = 'sentence'
tqdm.pandas(desc="Converting to vector embeddings")
df_kartverket[column_to_vectorise + '_vector'] = df_kartverket[column_to_vectorise].progress_apply(compute_embeddings)
df_kartverket.head()

Converting to vector embeddings: 100%|██████████| 904/904 [00:38<00:00, 23.62it/s]


,schema,uuid,id,hierarchyLevel,title,datasetcreationdate,abstract,keyword,geoBox,Constraints,...,LegalConstraints,temporalExtent,image,responsibleParty,link,metadatacreationdate,productInformation,parentId,sentence,sentence_vector
0,iso19139,7a62f16f-9aeb-4c39-bf5f-e710232fa366,37228,software,Artsfunn,,Datasettet inneholder stedfestet informasjon a...,Natur###Norge###Svalbard###lav###karplanter###...,2###33###57###81,",",...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://editor.geonorge.no/thumbnails/7a62f16f...,Frank HansenNorsk institutt for naturforskning...,###https://ipt.nina.no/,2021-03-24 00:00:00,Produktspesifikasjon###Produktark###Produktsid...,,Datasettet inneholder stedfestet informasjon a...,"[-0.4274571, 0.11139302, 0.114035614, 0.014649..."
1,iso19139,7a62f16f-9aeb-4c39-bf5f-e710232fa366,37228,software,Artsfunn,,Datasettet inneholder stedfestet informasjon a...,Natur###Norge###Svalbard###lav###karplanter###...,2###33###57###81,",",...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://editor.geonorge.no/thumbnails/7a62f16f...,Frank HansenNorsk institutt for naturforskning...,###https://ipt.nina.no/,2021-03-24 00:00:00,Produktspesifikasjon###Produktark###Produktsid...,,Funndataene følger datastandarden Darwin Core ...,"[0.111423515, 0.1313381, -0.2433284, 0.0981122..."
2,iso19139,79013154-92ee-4647-b160-925cbc148601,21400,dataset,Hav og is - Iskart (shapefil),,Istjenesten ved Meteorologisk institutt utarbe...,Oceanographic geographical features###Inspire#...,2.00###33.00###57.00###72.00,Bruksbegrensninger Ingen begrensninger på bruk...,...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,Meteorologisk instituttistjenesten@met.no###Me...,###http://polarview.met.no/,2023-11-15 00:00:00,Produktspesifikasjon###Produktark###Produktsid...,,Istjenesten ved Meteorologisk institutt utarbe...,"[-0.25687775, 0.50113523, -0.08046141, -0.2081..."
3,iso19139,79013154-92ee-4647-b160-925cbc148601,21400,dataset,Hav og is - Iskart (shapefil),,Istjenesten ved Meteorologisk institutt utarbe...,Oceanographic geographical features###Inspire#...,2.00###33.00###57.00###72.00,Bruksbegrensninger Ingen begrensninger på bruk...,...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,Meteorologisk instituttistjenesten@met.no###Me...,###http://polarview.met.no/,2023-11-15 00:00:00,Produktspesifikasjon###Produktark###Produktsid...,,Kartene viser iskonsentrasjon og er basert på ...,"[-0.34487146, 0.46353433, -0.16695763, -0.2243..."
4,iso19139,79013154-92ee-4647-b160-925cbc148601,21400,dataset,Hav og is - Iskart (shapefil),,Istjenesten ved Meteorologisk institutt utarbe...,Oceanographic geographical features###Inspire#...,2.00###33.00###57.00###72.00,Bruksbegrensninger Ingen begrensninger på bruk...,...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,Meteorologisk instituttistjenesten@met.no###Me...,###http://polarview.met.no/,2023-11-15 00:00:00,Produktspesifikasjon###Produktark###Produktsid...,,I tillegg til disse dataene brukes visuelle og...,"[-0.35273665, 0.063657, 0.003657626, 0.0274507..."


## 2. Creation of Milvus collection for metadata
Before creating a collection. The database should be examined for existing collections, to avoid forever hanging collections.

In [9]:
from pymilvus import list_collections, drop_collection, connections, MilvusException

server_host = 'ebjerk.no'
server_port = '19530'

connections.connect(host=server_host, port=server_port)
print(f'Current collections in database: {list_collections()}')

collections_to_drop = ['search_article_in_medium', 'your_collection_name', 'search_article_in_medium_salesforce']

for collection in collections_to_drop:
    if len(list_collections()) == 0:
        print(f'Database has no collections to remove.')
        break
    try:
        drop_collection(collection)
    except MilvusException:
        print(f'Could not drop collection: {collection}')

RPC error: [drop_collection], <MilvusException: (code=1, message=DescribeCollection failed: can't find collection: search_article_in_medium)>, <Time:{'RPC start': '2024-02-27 08:45:06.995162', 'RPC error': '2024-02-27 08:45:07.030559'}>
RPC error: [drop_collection], <MilvusException: (code=1, message=DescribeCollection failed: can't find collection: your_collection_name)>, <Time:{'RPC start': '2024-02-27 08:45:07.067512', 'RPC error': '2024-02-27 08:45:07.102182'}>


Current collections in database: ['kartverket_metadata']
Could not drop collection: search_article_in_medium
Could not drop collection: your_collection_name


RPC error: [drop_collection], <MilvusException: (code=1, message=DescribeCollection failed: can't find collection: search_article_in_medium_salesforce)>, <Time:{'RPC start': '2024-02-27 08:45:07.136942', 'RPC error': '2024-02-27 08:45:07.175644'}>


Could not drop collection: search_article_in_medium_salesforce


## 2. Creation of Milvus collection for metadata

In [10]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility


connections.connect(host=server_host, port=server_port)

def kartverket_create_milvus_collection(collection_name, vector_column, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)

    fields = [
            FieldSchema(name='schema', dtype=DataType.VARCHAR, max_length=100),  
            FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=False),
            FieldSchema(name='uuid', dtype=DataType.VARCHAR, max_length=100), 
            FieldSchema(name='hierarchyLevel', dtype=DataType.VARCHAR, max_length=100),    
            #FieldSchema(name='hierarchyLevel_vector', dtype=DataType.FLOAT_VECTOR, dim=dim), 
            FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=100),   
            #FieldSchema(name="title_vector", dtype=DataType.FLOAT_VECTOR, dim=dim),

            FieldSchema(name='datasetcreationdate', dtype=DataType.VARCHAR, max_length=500),    
            FieldSchema(name='abstract', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='abstract_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='keyword', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='keyword_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='geoBox', dtype=DataType.VARCHAR, max_length=100),    
            #FieldSchema(name='geoBox_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),    
            FieldSchema(name='Constraints', dtype=DataType.VARCHAR, max_length=1000),   
            #FieldSchema(name='Constraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   

            FieldSchema(name='SecurityConstraints', dtype=DataType.VARCHAR, max_length=500),   
            #FieldSchema(name='SecurityConstraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='LegalConstraints', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='LegalConstraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='temporalExtent', dtype=DataType.VARCHAR, max_length=100),    
            ##FieldSchema(name='temporalExtent_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),    
            FieldSchema(name='image', dtype=DataType.VARCHAR, max_length=1000),    
            FieldSchema(name='responsibleParty', dtype=DataType.VARCHAR, max_length=500),   
            #FieldSchema(name='responsibleParty_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   

            FieldSchema(name='link', dtype=DataType.VARCHAR, max_length=500),    
            #FieldSchema(name='metadatacreationdate', dtype=DataType.VARCHAR, max_length=500), # SUS field, encoding error?
            ##FieldSchema(name='metadatacreationdate_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),    
            FieldSchema(name='productInformation', dtype=DataType.VARCHAR, max_length=1000),   
            #FieldSchema(name='productInformation_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='parentId', dtype=DataType.VARCHAR, max_length=100),   


            FieldSchema(name='sentence', dtype=DataType.VARCHAR, max_length=1000),   
            FieldSchema(name='sentence_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   

    ]
    schema = CollectionSchema(fields=fields, description='search text')
    collection_columns = [field_schema.name for field_schema in schema.fields]
    collection = Collection(name=collection_name, schema=schema)
    
    index_params = {
        'metric_type': "L2",
        'index_type': "IVF_FLAT",
        'params': {"nlist": 2048}
    }
    collection.create_index(field_name=vector_column, index_params=index_params)
    return collection, collection_columns

kartverket_collection, collection_columns = kartverket_create_milvus_collection('kartverket_metadata', column_to_vectorise + '_vector', 1024)#768)

### 2.1 Creation of dataframe subset to exclude columns with complex data types

In [11]:
# Loads the correct columns from the dataframe in the correct order for milvus
df_kartverket_slice = df_kartverket[collection_columns]
df_kartverket_slice

,schema,id,uuid,hierarchyLevel,title,datasetcreationdate,abstract,keyword,geoBox,Constraints,SecurityConstraints,LegalConstraints,temporalExtent,image,responsibleParty,link,productInformation,parentId,sentence,sentence_vector
0,iso19139,37228,7a62f16f-9aeb-4c39-bf5f-e710232fa366,software,Artsfunn,,Datasettet inneholder stedfestet informasjon a...,Natur###Norge###Svalbard###lav###karplanter###...,2###33###57###81,",",#########,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://editor.geonorge.no/thumbnails/7a62f16f...,Frank HansenNorsk institutt for naturforskning...,###https://ipt.nina.no/,Produktspesifikasjon###Produktark###Produktsid...,,Datasettet inneholder stedfestet informasjon a...,"[-19.053547, 6.606391, 6.217847, 9.046252, -9...."
1,iso19139,37228,7a62f16f-9aeb-4c39-bf5f-e710232fa366,software,Artsfunn,,Datasettet inneholder stedfestet informasjon a...,Natur###Norge###Svalbard###lav###karplanter###...,2###33###57###81,",",#########,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://editor.geonorge.no/thumbnails/7a62f16f...,Frank HansenNorsk institutt for naturforskning...,###https://ipt.nina.no/,Produktspesifikasjon###Produktark###Produktsid...,,Funndataene følger datastandarden Darwin Core ...,"[-24.275457, 8.014599, 4.697033, 5.890971, -4...."
2,iso19139,21400,79013154-92ee-4647-b160-925cbc148601,dataset,Hav og is - Iskart (shapefil),,Istjenesten ved Meteorologisk institutt utarbe...,Oceanographic geographical features###Inspire#...,2.00###33.00###57.00###72.00,Bruksbegrensninger Ingen begrensninger på bruk...,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,Meteorologisk instituttistjenesten@met.no###Me...,###http://polarview.met.no/,Produktspesifikasjon###Produktark###Produktsid...,,Istjenesten ved Meteorologisk institutt utarbe...,"[-19.681519, 7.9921904, 5.2225614, 7.1261663, ..."
3,iso19139,21400,79013154-92ee-4647-b160-925cbc148601,dataset,Hav og is - Iskart (shapefil),,Istjenesten ved Meteorologisk institutt utarbe...,Oceanographic geographical features###Inspire#...,2.00###33.00###57.00###72.00,Bruksbegrensninger Ingen begrensninger på bruk...,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,Meteorologisk instituttistjenesten@met.no###Me...,###http://polarview.met.no/,Produktspesifikasjon###Produktark###Produktsid...,,Kartene viser iskonsentrasjon og er basert på ...,"[-19.882036, 9.381783, 7.9555464, 8.613085, -6..."
4,iso19139,21400,79013154-92ee-4647-b160-925cbc148601,dataset,Hav og is - Iskart (shapefil),,Istjenesten ved Meteorologisk institutt utarbe...,Oceanographic geographical features###Inspire#...,2.00###33.00###57.00###72.00,Bruksbegrensninger Ingen begrensninger på bruk...,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,Meteorologisk instituttistjenesten@met.no###Me...,###http://polarview.met.no/,Produktspesifikasjon###Produktark###Produktsid...,,I tillegg til disse dataene brukes visuelle og...,"[-21.088411, 4.6286764, 7.451848, 8.84124, -4...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,iso19139,50171,982c1c55-b96c-4021-aad1-a4a2d1bdbdee,dataset,Arealplanområder Svalbard,,Arealplaner på Svalbard følger et forenklet sy...,Annet###fellesDatakatalog###Plan###Svalbard###...,11.716018###17.082964###77.833805###78.963267,Bruksbegrensninger Oppgje alltid Sysselmannen ...,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://editor.geonorge.no/thumbnails/982c1c55...,Sysselmannen på Svalbardsbr@sysselmannen.no###...,,Produktspesifikasjon###Produktark###Produktsid...,,I de andre arealplanområdene er Sysselmannen p

### 2.2 Insert the subset dataframe data into Milvus collection

In [12]:
from towhee import ops, pipe, DataCollection

insert_pipe_kartverket = (pipe.input('df_kartverket_slice')
                   .flat_map('df_kartverket_slice', 'data', lambda df: df.values.tolist())
                   .map('data', 'res', ops.ann_insert.milvus_client(host=server_host, 
                                                                    port=server_port,
                                                                    collection_name='kartverket_metadata'))
                   .output('res')
)

%time _ = insert_pipe_kartverket(df_kartverket_slice)


kartverket_collection.load()
kartverket_collection.num_entities

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


CPU times: user 3.87 s, sys: 389 ms, total: 4.25 s
Wall time: 44 s


904

## 3. Prepare pipe that accepts queries

In [136]:
import numpy as np
# Variables specifying what column and collection to perform ANN comparrison against
vector_columns = [column_to_vectorise + '_vector']
collection_name = 'kartverket_metadata'

print(df_kartverket.columns)
# What columns to return for view
response_output = [
       'id', 'id', 'title', # required to have two 'id'
       'sentence',
       'abstract', 'keyword', 'Constraints',
       'SecurityConstraints', 'LegalConstraints',
       'responsibleParty', 'productInformation', 'parentId'
]


demo_pipe = (pipe.input('query')
                    .map('query', 'vec', ops.text_embedding.dpr(model_name=chosen_model))
                    .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
                    .flat_map('vec', ('id', 'score', 'id', 'title', # required to have two 'id'
                            'sentence', 'abstract', 'keyword', 'Constraints',
                            'SecurityConstraints', 'LegalConstraints',
                            'responsibleParty', 'productInformation', 'parentId'), 
                                       ops.ann_search.milvus_client(host=server_host, 
                                                                    port=server_port,
                                                                    collection_name=collection_name,
                                                                    vector_field=vector_columns,
                                                                    output_fields=response_output, 
                                                                    limit=5))  
                    .output(*['query', 'score'], *response_output)
               )

Index(['schema', 'uuid', 'id', 'hierarchyLevel', 'title',
       'datasetcreationdate', 'abstract', 'keyword', 'geoBox', 'Constraints',
       'SecurityConstraints', 'LegalConstraints', 'temporalExtent', 'image',
       'responsibleParty', 'link', 'metadatacreationdate',
       'productInformation', 'parentId', 'abstract_vector'],
      dtype='object')


2024-02-23 13:25:38,736 - 8067764224 - dpr.py-dpr:34 - ERROR: Fail to load tokenizer by name: ltg/norbert3-large


RuntimeError: Node-text-embedding/dpr-0 runs failed, error msg: Create text-embedding/dpr-0 operator text-embedding/dpr:main with args None and kws {'model_name': 'ltg/norbert3-large'} failed, err: stat: path should be string, bytes, os.PathLike or integer, not NoneType, Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/towhee/runtime/nodes/node.py", line 88, in initialize
    self._op = self._op_pool.acquire_op(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/towhee/runtime/operator_manager/operator_pool.py", line 106, in acquire_op
    op = self._op_loader.load_operator(hub_op_id, op_args, op_kws, tag, latest)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/towhee/runtime/operator_manager/operator_loader.py", line 154, in load_operator
    op = factory(function, arg, kws, tag, latest)
  File "/Library/Frameworks/Python.framework/Vers

## 3.1 Query against metadata collection

In [ ]:
kartverket_question1 = 'Hvor kan jeg finne datasett har informasjon om oljeplatformer?'#'I tillegg til disse dataene brukes visuelle og infraroede data fra MODIS og NOAA satellitter.'#'Roerledningssystem med tilhoerende installasjoner, utgjoer transportsystemene for petroleum fra Norsk kontinentalsokkel'
print(f'\n"{kartverket_question1}" search:')
res_kartverket1 = demo_pipe(kartverket_question1)
DataCollection(res_kartverket1).show()

## 4. Format response into GPT API request
After the query is vectorised, and then compared against the database's vector column to find the most simmilar result(s). The most similar result's metadata can be formatted into a instruction with example for GPT. This way, we can instruct GPT how to answer the query by providing a demonstration. Before prompting with the query's current context, and the instruction to answer.

![alt text](image.png)

In [ ]:
metadata_demonstration = '''query	score	id	id	title	abstract	keyword	Constraints	SecurityConstraints	LegalConstraints	responsibleParty	productInformation	parentId
Hvor er bomberom?	118.3987045288086	22788	22788	Magasin	Database over regulerte innsjøer. Egenskapsdata er vassdragsnr., magasinnr., navn, laveste og høyeste regulerte vannstand (m.o.h...	Annet###Åpne data###Norge digitalt###modellbaserteVegprosjekter###fellesDatakatalog###Energi###Norge fastland###magasin###vannkr...	Bruksbegrensninger Ingen ###Ingen	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Seming Haakon SkauNorges vassdrags- og energidirektoratgisstotte@nve.no###NVE - Seksjon for geomatikk og dataanalyse/IGDNorges v...	Produktspesifikasjon###Produktark###Produktside###Tegnforklaring###dekningsoversikt###hjelp###dekningsoversikt rutenett###deknin...
Hvor er bomberom?	123.4372329711914	21492	21492	Bunnsedimenter (kornstørrelse), detaljert	Datasettet viser kornstørrelsessammensetning i sjøbunnssedimentenes øvre del (øverste 0-10 cm av sjøbunnen). I egenskapstabellen...	Sea regions###Inspire###Norge digitalt###geodataloven###Mareano###ØkologiskGrunnkart###MarineGrunnkart###fellesDatakatalog###Geo...	Bruksbegrensninger Detaljnivået på datasettet tilsier bruk innenfor kartmålestokken: 1:20.000 - 1:150.000. ###Detaljnivået på d...	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Aave LeplandNorges geologiske undersøkelseDataadministrator4773904000Leiv Eirikssons vei 39TrondheimAave.Lepland@ngu.nohttp://ww...	https://register.geonorge.no/produktspesifikasjoner/bunnsedimenter-kornstørrelseProduktspesifikasjon###https://register.geonorge...
Hvor er bomberom?	124.96815490722656	68030	68030	Bunnsedimenter (kornstørrelse), oversikt	Datasettet viser kornstørrelsessammensetning i sjøbunnssedimentenes øvre del. Kornstørrelsesdata er basert på analyser av sjøbun...	Sea regions###Inspire###Norge digitalt###geodataloven###Mareano###fellesDatakatalog###Geologi###Norge###Nordsjøen###Norskehavet#...	Bruksbegrensninger Detaljnivået på datasettet tilsier bruk innenfor kartmålestokken: 1:2000.000 - 1:10.000.000 ###Detaljnivået ...	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Aave LeplandNorges geologiske undersøkelseDataadministrator4773904000Leiv Eirikssons vei 39TrondheimAave.Lepland@ngu.nohttp://ww...	https://register.geonorge.no/produktspesifikasjoner/bunnsedimenter-kornstørrelseProduktspesifikasjon###https://register.geonorge...
Hvor er bomberom?	125.0992431640625	75832	75832	Vegetasjon - Naturtyper	Kommunvise kartlag i datasettet Vegetasjon som viser naturtyper som er viktig for biologisk mangfold og som helt eller delvis ka...	Land cover###Åpne data###Norge digitalt###fellesDatakatalog###Landbruk###Natur###Norge fastland###Vegetasjon###Arealdekke###Vege...	Bruksbegrensninger Ingen ###Ingen	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Ingvild NystuenNorsk institutt for bioøkonomiSeksjonsleder64 94 80 0064 94 80 01Raveien 9ÅsAkersjusPostboks 115, 1430 ÅsNorgegis...	http://www.skogoglandskap.no/seksjoner/nedlastingsinformasjonNedlastingsinformasjon###http://www.skogoglandskap.no/kart/vegetasj...
Hvor er bomberom?	125.26884460449219	69607	69607	Relativ bunnhardhet, åpne data	Relativ bunnhardhet er rasterdata som viser reflektivitetstyrke fra sjøbunnen. Reflektivitetsstyrke sier noe om sjøbunnens akust...	Geology###Åpne data###Norge digitalt###MarineGrunnkart###modellbaserteVegprosjekter###fellesDatakatalog###Geologi###Norge###Bare...	Bruksbegrensninger Ingen begrensninger på bruk er oppgitt. ###Ingen begrensninger på bruk er oppgitt.No conditions apply	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Aave LeplandNorges geologiske undersøkelseDataadministrator, Maringeologi, NGU4773904000Leiv Eirikssons vei 39TrondheimAave.Lepl...	Produktspesifikasjon###https://register.geonorge.no/register/versjoner/produktark/norges-geologiske-undersokelse/relativ-bunnhar...'''
metadata_link = 'https://ipt.nina.no/'

query = f'Finn dataset med magasin'
instruction_gpt = f'Skriv en respons som finner det mest korresponderende datasettet fra metadata for spørringen:'
demonstration = f'Spørring: Hvor er bomberom?\nMetadata: {metadata_demonstration}\nRespons: Ut i fra spørringen ser det ut som at du leter etter datasettet Tilfluktsrom (nedlastningslink: {metadata_link}). Dette datsettet inneholder informasjon om lokasjonene til alle offentlige tilfluktsrom (bomberom) i Norge'
current_query_instruction = f'Spørring:{query}\nMetadata: {metadata_demonstration}\nRespons:'
API_text = f'{instruction_gpt}\n{demonstration}\n\n{current_query_instruction}'
print(API_text)

# Testing

In [8]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm

# Load tokenizer and model
norBert_base = 'ltg/norbert3-base'
norBert_large = 'ltg/norbert3-large'
chosen_model = norBert_large
tokenizer = AutoTokenizer.from_pretrained(chosen_model)
model = AutoModel.from_pretrained(chosen_model, trust_remote_code=True)

# Ensure model is in evaluation mode
model.eval()

# Function that computes the column text embeddings manually
def compute_embeddings_test(text):
    MAX_TOKENS = 512 
    inputs = tokenizer(text, return_tensors="pt", max_length=MAX_TOKENS, truncation=True)
    with torch.no_grad():
        output = model(**inputs)
    embeddings = output.last_hidden_state
    cls_embeddings = embeddings[:, 0, :]

    # Normalize the embeddings
    norm = torch.linalg.norm(cls_embeddings, dim=1, keepdim=True)
    normalized_embeddings = cls_embeddings / norm

    return normalized_embeddings.squeeze().numpy()


# Load your dataset
df_kartverket = pd.read_excel(expanded_dataset_file)

# Recast columns to correct datatypes
recast_to_string = ['datasetcreationdate', 'metadatacreationdate']
df_kartverket[recast_to_string] = df_kartverket[recast_to_string].astype('object')

# Fill NaN values with an empty string
df_kartverket.fillna('', inplace=True)

# Process each column and create new columns for embeddings
column_to_vectorise = 'sentence'
tqdm.pandas(desc="Converting to vector embeddings")
df_kartverket[column_to_vectorise + '_vector'] = df_kartverket[column_to_vectorise].progress_apply(compute_embeddings_test)

# Display the first few rows of the dataframe
df_kartverket.head()

Converting to vector embeddings: 100%|██████████| 904/904 [02:03<00:00,  7.35it/s]


,schema,uuid,id,hierarchyLevel,title,datasetcreationdate,abstract,keyword,geoBox,Constraints,...,LegalConstraints,temporalExtent,image,responsibleParty,link,metadatacreationdate,productInformation,parentId,sentence,sentence_vector
0,iso19139,7a62f16f-9aeb-4c39-bf5f-e710232fa366,37228,software,Artsfunn,,Datasettet inneholder stedfestet informasjon a...,Natur###Norge###Svalbard###lav###karplanter###...,2###33###57###81,",",...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://editor.geonorge.no/thumbnails/7a62f16f...,Frank HansenNorsk institutt for naturforskning...,###https://ipt.nina.no/,2021-03-24 00:00:00,Produktspesifikasjon###Produktark###Produktsid...,,Datasettet inneholder stedfestet informasjon a...,"[-19.053547, 6.606391, 6.217847, 9.046252, -9...."
1,iso19139,7a62f16f-9aeb-4c39-bf5f-e710232fa366,37228,software,Artsfunn,,Datasettet inneholder stedfestet informasjon a...,Natur###Norge###Svalbard###lav###karplanter###...,2###33###57###81,",",...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://editor.geonorge.no/thumbnails/7a62f16f...,Frank HansenNorsk institutt for naturforskning...,###https://ipt.nina.no/,2021-03-24 00:00:00,Produktspesifikasjon###Produktark###Produktsid...,,Funndataene følger datastandarden Darwin Core ...,"[-24.275457, 8.014599, 4.697033, 5.890971, -4...."
2,iso19139,79013154-92ee-4647-b160-925cbc148601,21400,dataset,Hav og is - Iskart (shapefil),,Istjenesten ved Meteorologisk institutt utarbe...,Oceanographic geographical features###Inspire#...,2.00###33.00###57.00###72.00,Bruksbegrensninger Ingen begrensninger på bruk...,...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,Meteorologisk instituttistjenesten@met.no###Me...,###http://polarview.met.no/,2023-11-15 00:00:00,Produktspesifikasjon###Produktark###Produktsid...,,Istjenesten ved Meteorologisk institutt utarbe...,"[-19.681519, 7.9921904, 5.2225614, 7.1261663, ..."
3,iso19139,79013154-92ee-4647-b160-925cbc148601,21400,dataset,Hav og is - Iskart (shapefil),,Istjenesten ved Meteorologisk institutt utarbe...,Oceanographic geographical features###Inspire#...,2.00###33.00###57.00###72.00,Bruksbegrensninger Ingen begrensninger på bruk...,...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,Meteorologisk instituttistjenesten@met.no###Me...,###http://polarview.met.no/,2023-11-15 00:00:00,Produktspesifikasjon###Produktark###Produktsid...,,Kartene viser iskonsentrasjon og er basert på ...,"[-19.882036, 9.381783, 7.9555464, 8.613085, -6..."
4,iso19139,79013154-92ee-4647-b160-925cbc148601,21400,dataset,Hav og is - Iskart (shapefil),,Istjenesten ved Meteorologisk institutt utarbe...,Oceanographic geographical features###Inspire#...,2.00###33.00###57.00###72.00,Bruksbegrensninger Ingen begrensninger på bruk...,...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,Meteorologisk instituttistjenesten@met.no###Me...,###http://polarview.met.no/,2023-11-15 00:00:00,Produktspesifikasjon###Produktark###Produktsid...,,I tillegg til disse dataene brukes visuelle og...,"[-21.088411, 4.6286764, 7.451848, 8.84124, -4...."


In [13]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from pymilvus import connections

# Function that computes the query embeddings manually
def compute_query_embeddings_test(query):
    inputs = tokenizer(query, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        output = model(**inputs)
    embeddings = output.last_hidden_state
    cls_embeddings = embeddings[:, 0, :]

    # Normalize the embeddings
    norm = torch.linalg.norm(cls_embeddings, dim=1, keepdim=True)
    normalized_embeddings = cls_embeddings / norm

    return normalized_embeddings.numpy()


# Load tokenizer and model so they can be used for encoding of queries into embeddings
norBert_base = 'ltg/norbert3-base'
norBert_large = 'ltg/norbert3-large'
chosen_model = norBert_large
tokenizer = AutoTokenizer.from_pretrained(chosen_model)
model = AutoModel.from_pretrained(chosen_model, trust_remote_code=True)

# Sets the model to evaluation mode
model.eval()  




# Connect to Milvus server
connections.connect(host=server_host, port=server_port)
collection = Collection(name='kartverket_metadata')


# Prepare the search parameters
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}

# Perform the search
query = "Hvor i norge kan man drive regnsdrift?"#"Hvilket datasett har oljeplatformer?"
query_embeddings = compute_query_embeddings_test(query)
query_embeddings = query_embeddings.astype("float32")  
response_output = [
       #'id', 'id', # required to have two 'id' 
        'title', 
        'sentence',
        'abstract', 'keyword', 'Constraints',
        'SecurityConstraints', 'LegalConstraints',
        'responsibleParty', 'productInformation', 'parentId'
]

results = collection.search(
    data=query_embeddings, 
    anns_field=column_to_vectorise + '_vector', 
    param=search_params, 
    limit=10, 
    output_fields=response_output
)

# Process search results
for result in results:
    for hit in result:
        hit_info = f"Query: '{query}', ID: {hit.id}, Score: {hit.score}, "
        hit_info += ", ".join([f"{field}: {hit.entity.get(field)}" for field in response_output])
        print(hit_info)


2024-02-27 08:46:10,138 - 8067764224 - decorators.py-decorators:107 - ERROR: Unexcepted error: [search], `search_data` value [-15.101511    5.1437035   2.570294  ...  -3.8019092   9.49781
   2.8618627] is illegal, <Time: {'RPC start': '2024-02-27 08:46:10.136830', 'Exception': '2024-02-27 08:46:10.138390'}>


ParamError: `search_data` value [-15.101511    5.1437035   2.570294  ...  -3.8019092   9.49781
   2.8618627] is illegal

In [14]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from pymilvus import connections, Collection

# Function that computes the query embeddings manually
def compute_query_embeddings(query):
    inputs = tokenizer(query, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        output = model(**inputs)
    embeddings = output.last_hidden_state
    cls_embeddings = embeddings[:, 0, :]

    return cls_embeddings.squeeze().numpy()


def perform_milvus_search(query, collection_name, server_host, server_port, vectorised_column, response_output, limit=10):
    # Connect to Milvus server
    connections.connect(host=server_host, port=server_port)
    collection = Collection(name=collection_name)

    # Prepare the search parameters
    search_params = {
        "metric_type": "L2",
        "params": {"nprobe": 10},
    }

    # Compute query embeddings
    query_embeddings = compute_query_embeddings(query)
    query_embeddings = query_embeddings.astype("float32")


    # Perform the search
    results = collection.search(
        data=query_embeddings, 
        anns_field=vectorised_column, 
        param=search_params, 
        limit=limit, 
        output_fields=response_output
    )

    # Process search results
    for result in results:
        for hit in result:
            hit_info = f"Query: '{query}', ID: {hit.id}, Score: {hit.score}, "
            hit_info += ", ".join([f"{field}: {hit.entity.get(field)}" for field in response_output])
            print(hit_info)
    
    # TODO this function should return the responses in some format containing info about each response if they should be utilized further



## Preparation of model for encoding of queries

In [164]:

# Load tokenizer and model so they can be used for encoding of queries into embeddings
norBert_base = 'ltg/norbert3-base'
norBert_large = 'ltg/norbert3-large'
chosen_model = norBert_large
tokenizer = AutoTokenizer.from_pretrained(chosen_model)
model = AutoModel.from_pretrained(chosen_model, trust_remote_code=True)

# Sets the model to evaluation mode
model.eval()  

NorbertModel(
  (embedding): Embedding(
    (word_embedding): Embedding(50000, 1024)
    (word_layer_norm): LayerNorm((1024,), eps=1e-07, elementwise_affine=False)
    (dropout): Dropout(p=0.1, inplace=False)
    (relative_layer_norm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
  )
  (transformer): Encoder(
    (layers): ModuleList(
      (0-23): 24 x EncoderLayer(
        (attention): Attention(
          (in_proj_qk): Linear(in_features=1024, out_features=2048, bias=True)
          (in_proj_v): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (pre_layer_norm): LayerNorm((1024,), eps=1e-07, elementwise_affine=False)
          (post_layer_norm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (mlp): FeedForward(
          (mlp): Sequential(
            (0): LayerNorm((1024,), eps=1e-07, elementwise_affine=Fa

In [16]:
# Query using the function
# These work somewhat
query1 = 'Hvor i norge kan man drive reinsdrift?'
query2 = 'Hvor er det oljeplatformer i norge?'
query3 = 'Hvor har forsvaret militærbaser?'
query4 = 'Hvor er det jernmalm i Norge?'


vectorised_column = column_to_vectorise + '_vector'
# Response output fields
response_output = [
    'sentence', 'title',  'abstract', 'keyword', 'Constraints',
    'SecurityConstraints', 'LegalConstraints',
    'responsibleParty', 'productInformation', 'parentId'
]
perform_milvus_search(query2, 'kartverket_metadata', server_host, server_port, vectorised_column, response_output)

Query: 'dflajdfaøsfnas fewq nfou wef aoisdnfm øiajfen k', ID: 67972, Score: 0.0017401196528226137, sentence: mineralloven § 23 og §§ 33 - 34., title: Bergrettigheter, abstract: Dataene viser områder som er belagt med bergrettigheter for statens mineraler, som definert av Lov om erverv og utvinning av mineralressurser (mineralloven) §7. Datasettet inneholder definerte områder for undersøkelses (UN)- og utvinningsretter (UT). Det kan være utstedt flere bergrettigheter for samme geografiske område; datasettet har overlappende flater, men de individuelle rettighetene har forskjellig prioritet. Første prioritet gjelder basert på mottatt dato, beskrevet i attributtet InnsendtDato. Direktoratet for mineralforvaltning med Bergmesteren for Svalbard (DMF) er eier av datasettet. En undersøkelsesrett på statens mineraler tildeles som en rett på et bestemt område, og ikke som en rettighet til en bestemt forekomst. Den som har undersøkelsesrett med best prioritet, dvs. den som søkte om retten først,